# N,P, K balances

This notebook balances the NPK inputs and outputs assuming that any deficit is covered by mineral fertilizer and any excess is reduced from mineral fertilizer and N fixation.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
#In this cell the scenario is chosen. This can either be defined here or in the RUN_ALL.ipynb
#If defined here unmark the scen = line and select the scenario you will run.
#If defined in RUN_ALL.ipynb unmark the %store -r scen line.
#Scenario names are:
#S0_MinFert > N,P,K demand met by mineral fertilizers only
#S0_struvite_P > P demand met by struvite, N demand met by struvite and mineral fertilizer and K demand met by mineral fertilizer
#S0_compost > N,P,K supplied by compost produced in the AMB, remaining N,P,K demand met by mineral fertilizer
#S0_Ammon_salts > N supplied from recovered ammonium salts and from mineral fertilizers, P from struvite and K from mineral fertilizer
scen = 'S0_MinFert'
#%store -r scen

In [ ]:
print(scen)

In [ ]:
%store -r URBAG_map
%store -r d_conv_emiss

%store -r Manure_N
%store -r Manure_P2O5

%store -r N_agriwaste_to_field

%store -r Mineral_fert_N
%store -r Mineral_fert_P2O5
%store -r Mineral_fert_K2O
%store -r Ammonium_salts

%store -r Struvite_P2O5
%store -r Struvite_N
%store -r Struvite_K2O

%store -r Compost_P2O5
%store -r Compost_N
%store -r Compost_K2O

In [ ]:
#Crops that have N fixation
N_fixation_crop = ['ALFALS',
                   'CIGRONS',
                   'FAVES I FAVONS',
                   'GARROFER',
                   'Garroferars',
                   'Garroferars abandonats - prats en zones agrÃƒÂ\xadcoles',
                   'Garroferars abandonats - prats en zones agrícoles',
                   'Garroferars en bancals',
                   'PESOLS',
                   'TREPADELLA']

In [ ]:
len_URBAG_map= len(URBAG_map['Voronoi_1'])
print(f"This is the amount of plots in the map in this notebook: {len_URBAG_map}")

In [ ]:
#Others
NH3_N_to_NH3 = 17/14
NH3_to_NH3_N = 14/17
N2O_N_to_N2O = 44/28
N2O_to_N2O_N = 28/44
NO2_N_to_NO2 = 46/14
NO2_to_NO2_N = 14/46
NO3_N_to_NO3 = 62/14
NO3_to_NO3_N = 14/62
C_to_CO2 = 44/12

conv6 = np.repeat(NH3_to_NH3_N, len_URBAG_map) #Conversion from NH3 to NH3-N
conv7 = np.repeat(NO2_to_NO2_N, len_URBAG_map) #Conversion NO2 to NO2-N
conv8 = np.repeat(N2O_to_N2O_N, len_URBAG_map) #Conversion N2O to NO2-N
conv9 = np.repeat(NO3_to_NO3_N, len_URBAG_map) #Conversion NO3 to NO3-N

#constants to change emissions of nitrate from Nitrate-N to Nitrate
NO3_N_to_NO3 = 62/14
P_to_P2O5 = 2.29
P2O5_to_P = 0.4364
P_to_PO43 = 1/0.3261
PO43_to_P = 0.3261
conv4  = np.repeat(NO3_N_to_NO3, len_URBAG_map) #Conversion from NO3-N to NO3
conv9_1  = np.repeat(P_to_P2O5, len_URBAG_map) #Conversion P to P2O5
conv10 = np.repeat(P2O5_to_P, len_URBAG_map) #Conversion P2O5 to P2O5-P
conv11 = np.repeat(P_to_PO43, len_URBAG_map) #Conversion phosphate-P to phosphate
conv12 = np.repeat(PO43_to_P, len_URBAG_map) #Conversion phosphate to phosphate-P

In [ ]:
N_balance = pd.DataFrame(URBAG_map)
N_balance.head()

## Nitrogen Balance

### N Fixation

In [ ]:
#List with all voronoi categories for all parcels
Fresh_matter_Voronoi_list = URBAG_map['Voronoi_1'].tolist()

#List with total mineral N inputs for all parcels, assumed to be the same as N removed with the harvest
Fresh_matter_N_harv_list = URBAG_map['kgN/ha'].tolist()


In [ ]:
#Adding N Fixation for fixating crops and is equal to N removed with the harvest
N_fixation = []   
for row in range(0,np.shape(Fresh_matter_Voronoi_list)[0]):
    vor = Fresh_matter_Voronoi_list[row]
    if vor in N_fixation_crop:
        N_fixation.append(Fresh_matter_N_harv_list[row])
    else:
        N_fixation.append(0)

In [ ]:
len_N_fixation = len(N_fixation)
print(f"This is the amount of plots in the N fixation parameter: {len_N_fixation}")

### Total N emissions to air and water

In [ ]:
#Emissions as kg N/ha
NH3_N = d_conv_emiss['NH3_total_air'] * conv6
NOx_N = d_conv_emiss['NOx_total_air'] * conv7
N2O_N = d_conv_emiss['N2O_total_air'] * conv8
NO3_N = d_conv_emiss['NO3_total'] * conv9
Outputs_N_out = NH3_N + NOx_N + N2O_N + NO3_N

### Total N inputs

In [ ]:
#Inputs calculation [kg N/ha]
#N inputs incl. (mineral fertilizers, manure, residues returned to field) + N2 fixation by fixating crops + struvite inputs + Compost inputs
N_inputs_N_fix = Ammonium_salts + Mineral_fert_N + Manure_N + N_agriwaste_to_field + N_fixation + Struvite_N + Compost_N

In [ ]:
N_inputs_N_fix.describe()

In [ ]:
#Inputs minus outputs
#(mineral fertilizers + manure + residues returned to field + N fixation + Struvite + Compost) - (N removal with harvest + NH3 + NOx + N2O + NO3_base)
Balance_N = N_inputs_N_fix - (Outputs_N_out + Fresh_matter_N_harv_list)

In [ ]:
Balance_N.describe()

### N variables before balance

In [ ]:
#Variable with inputs, outputs and balance before adjustments
N_balance['N_fert'] = Mineral_fert_N  #Before balance this is equal to N remove with harvest
N_balance['N_manure'] = Manure_N
N_balance['N_residues'] = N_agriwaste_to_field
N_balance['N_Struvite'] = Struvite_N #Before balance this is equal to N remove with harvest
N_balance['N_Compost'] = Compost_N
N_balance['N_Ammonium_salts'] = Ammonium_salts
N_balance['N_in'] = Mineral_fert_N + Ammonium_salts + Manure_N + N_agriwaste_to_field + Struvite_N + Compost_N
N_balance['N_fix'] = N_fixation
N_balance['N_in_incl_N_fix'] = N_inputs_N_fix #>>>!!!ALL INPUTS incl. fix, struvite and compost
N_balance['N_NO3_base'] = NO3_N
N_balance['N_out'] = Outputs_N_out #Emissions sum NH3 + NOx + NO3 + N2O
N_balance['N_harv'] = Fresh_matter_N_harv_list #N removed with harvest
N_balance['N_out_incl_N_harv'] = Outputs_N_out + Fresh_matter_N_harv_list #>>>!!!ALL OUTPUTS
N_balance['N_Balance_incl_fix'] = Balance_N #N_inputs_N_fix - Outputs_N_4

In [ ]:
#Total kg N per yr from mineral fertilizer
np.sum(N_balance['Area']*(1/10000) * N_balance['N_fert'])/1000

In [ ]:
print("The data before balance is:")

In [ ]:
print(N_balance[['N_fert', 'N_manure', 'N_residues', 'N_Struvite', 'N_Compost',
       'N_Ammonium_salts', 'N_in', 'N_fix', 'N_in_incl_N_fix',
       'N_NO3_base', 'N_out', 'N_harv', 'N_out_incl_N_harv',
       'N_Balance_incl_fix']].describe())

In [ ]:
test = N_balance.loc[N_balance['N_Balance_incl_fix']>0]
test
#test[test["Fid"]==1772]

### N Balancing equations

In [ ]:
#Condition 1: Plots where the balance is larger than 0 i.e. inputs are larger than outputs
#Condition 2: Plots where the balance is smaller than 0 i.e. outputs are larger than inputs
cond1 = N_balance['N_Balance_incl_fix'] > 0
cond2 = N_balance['N_Balance_incl_fix'] <= 0

#For plots in condition 1:
#These should be plots with N fixation
#N fertilizer is reduced with the excess
#Further in the N_bln variable fertilizer is set to 0 where this balance leads to negative fertilizer use and fixation is reduced as well.
if scen == 'S0_MinFert':
    N_balance.loc[N_balance.index[cond1],'N_fert_1']     = N_balance.loc[N_balance.index[cond1],'N_fert'] - N_balance.loc[N_balance.index[cond1],'N_Balance_incl_fix']
elif scen == 'S0_struvite_P':
    N_balance.loc[N_balance.index[cond1],'N_fert_1']     = N_balance.loc[N_balance.index[cond1],'N_fert'] - N_balance.loc[N_balance.index[cond1],'N_Balance_incl_fix']
elif scen == 'S0_compost':
    N_balance.loc[N_balance.index[cond1],'N_fert_1']     = N_balance.loc[N_balance.index[cond1],'N_fert'] - N_balance.loc[N_balance.index[cond1],'N_Balance_incl_fix']
elif scen == 'S0_Ammon_salts':
    N_balance.loc[N_balance.index[cond1],'N_fert_1']     = N_balance.loc[N_balance.index[cond1],'N_fert'] - N_balance.loc[N_balance.index[cond1],'N_Balance_incl_fix']
else:
    pass

#For plots in condition 2:
#These should be most plots
#N fertilizer is increased with the difference
if scen == 'S0_MinFert':
    N_balance.loc[N_balance.index[cond2],'N_fert_1']     = N_balance.loc[N_balance.index[cond2],'N_fert'] + np.abs(N_balance.loc[N_balance.index[cond2],'N_Balance_incl_fix'])
elif scen == 'S0_struvite_P':
    N_balance.loc[N_balance.index[cond2],'N_fert_1']     = N_balance.loc[N_balance.index[cond2],'N_fert'] + np.abs(N_balance.loc[N_balance.index[cond2],'N_Balance_incl_fix'])
elif scen == 'S0_compost':
    N_balance.loc[N_balance.index[cond2],'N_fert_1']     = N_balance.loc[N_balance.index[cond2],'N_fert'] + np.abs(N_balance.loc[N_balance.index[cond2],'N_Balance_incl_fix'])
elif scen == 'S0_Ammon_salts':
    N_balance.loc[N_balance.index[cond2],'N_fert_1']     = N_balance.loc[N_balance.index[cond2],'N_fert'] + np.abs(N_balance.loc[N_balance.index[cond2],'N_Balance_incl_fix'])
else:
    pass

In [ ]:
N_balance['N_fert_1'].describe()

In [ ]:
np.min(N_balance['N_fert_1'])

In [ ]:
np.max(N_balance['N_fert_1'])

In [ ]:
np.max(N_balance['N_harv'])

In [ ]:
#plots with no removed N should only be with no crops in
pd.unique(N_balance.loc[N_balance['N_harv']==0]["Voronoi_1"])

### N variables after balance 

In [ ]:
URBAG_map.keys().values

In [ ]:
N_bln = pd.DataFrame(URBAG_map)

In [ ]:
N_bln.keys().values

In [ ]:
N_bln = pd.DataFrame(URBAG_map.drop(['TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR', 'DUN_GRUP',
        'TF', 'REG_CREAF', 'REG_CAT', 'TF_REG',
        '01_ETO', '02_ETO', '03_ETO', '04_ETO',
       '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO',
       '11_ETO', '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT',
       '05_PPT', '06_PPT', '07_PPT', '08_PPT', '09_PPT', '10_PPT',
       '11_PPT', '12_PPT', 
       'ID_SUBCUEN', 'N_SUBCUENC', 'SUBCUENCAS',
        'EF_Tier2',
       'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
       'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9',
       'Kc_b_Kc10', 'Kc_b_Kc11', 'Kc_b_Kc12', 'N_T', 'P_T', 'K_T',
       'Voronoi_ag', 'Vor_ag_Num', 'FoodTons_N', '01_CWR', '02_CWR',
       '03_CWR', '04_CWR', '05_CWR', '06_CWR', '07_CWR', '08_CWR',
       '09_CWR', '10_CWR', '11_CWR', '12_CWR', '01_PPTe', '02_PPTe',
       '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe', '07_PPTe', '08_PPTe',
       '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe', '01_ID', '02_ID',
       '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID', '09_ID',
       '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
       '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE',
       '11_WE', '12_WE', 'WE_m3_ha_yr', 'Manure', 'Manure_N',
       'Manure_P2O5', 'Agriwaste_toField', 'Agriwaste_burned',
       'N_agriwaste_to_field',],axis=1))
N_bln.head()

In [ ]:
N_bln.keys().values

In [ ]:
N_bln['N_Manure'] = Manure_N #Kg N/ha
N_bln['N_Fertiliser'] = np.where(N_balance['N_fert_1']<0,0,N_balance['N_fert_1'])  #Kg N/ha
N_bln['N_Residues'] = N_agriwaste_to_field  #Kg N/ha
N_bln['N_Struvite'] = N_balance['N_Struvite'] #Kg N/ha (struvite)
N_bln['N_Compost'] = N_balance['N_Compost'] #Kg N/ha (struvite)
N_bln['N_Ammonium_salts'] = N_balance['N_Ammonium_salts']
N_bln['N_In'] = N_bln['N_Manure'] + N_bln['N_Fertiliser'] + N_bln['N_Ammonium_salts'] + N_bln['N_Residues'] + N_bln['N_Struvite'] + N_bln['N_Compost']#Kg N/ha (mineral fertilizers + manure + residues returned to field + struvite + compost)
N_bln['N_Fixation'] = np.where(N_balance['N_fert_1']<0, N_fixation+N_balance['N_fert_1'], N_fixation) #N fixation for symbiotic crops only
N_bln['N_in_incl_N_fix'] = N_bln['N_In'] + N_bln['N_Fixation']
N_bln['N_NH3'] = NH3_N #Kg N/ha
N_bln['N_NOx'] = NOx_N #Kg N/ha
N_bln['N_N2O'] = N2O_N #Kg N/ha
N_bln['N_NO3'] = NO3_N #Kg N/ha
N_bln['N_Harvest'] = N_balance['N_harv']
N_bln['N_Out'] = N_bln['N_NH3'] + N_bln['N_NOx'] + N_bln['N_N2O'] + N_bln['N_NO3'] + N_bln['N_Harvest'] #Kg N/ha
N_bln['N_Balance'] = N_bln['N_in_incl_N_fix'] - N_bln['N_Out'] #Kg N/ha
N_bln['N_bln_check'] = np.where(N_bln['N_Balance'] >= 0, 'Enough', 'Deficient')

In [ ]:
#plot with max N fixation
N_bln.loc[N_bln['N_Fixation']==np.max(N_bln['N_Fixation'])]

In [ ]:
print("The data after balance is:")

In [ ]:
print(N_bln[['kgP2O5/ha', 'kgK2O/ha',
       'kgN/ha', 'N_Manure', 'N_Fertiliser', 'N_Residues', 'N_Struvite',
       'N_Compost', 'N_Ammonium_salts', 'N_In', 'N_Fixation',
       'N_in_incl_N_fix', 'N_NH3', 'N_NOx', 'N_N2O', 'N_NO3', 'N_Harvest',
       'N_Out', 'N_Balance', 'N_bln_check']].describe())

In [ ]:
test = N_bln.loc[N_bln['Fid']==1882]
test['N_Fertiliser']

In [ ]:
N_bln.loc[N_bln['N_Balance']>1]["N_Ammonium_salts"]

In [ ]:
N_bln["N_Fertiliser"].describe()

In [ ]:
#Is there any plot for which outputs are larger than inputs? If false no plot has more outputs.
(N_bln['N_Balance'] < 0).values.any()

In [ ]:
N_bln['N_Balance'].max()

In [ ]:
N_bln['N_Balance'].min()

In [ ]:
#Categories for which the balance is not completed because there is no data for N in harvest
pd.unique(N_bln[N_bln['N_Balance'].isna()]['Voronoi_1'])

In [ ]:
ascending = N_bln.sort_values('N_Balance')
pd.unique(ascending['N_Balance'])

In [ ]:
N_bln.keys()

In [ ]:
#total compost use in scenario after balance in kg N per year
N_compost_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_Compost'])/1000
N_compost_after_balance

In [ ]:
#total struvite use in scenario after balance in kg N per year
N_struvite_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_Struvite'])/1000
N_struvite_after_balance

In [ ]:
#total N mineral fertilizer use in scenario after balance in kg N per year
N_MinFert_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_Fertiliser'])/1000
N_MinFert_after_balance

In [ ]:
#total N ammonium salts use in scenario after balance in kg N per year
N_AmmonSalts_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_Ammonium_salts'])/1000
N_AmmonSalts_after_balance

In [ ]:
#total N fixation in scenario after balance in kg N per year
N_Fixation_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_Fixation'])/1000
N_Fixation_after_balance

In [ ]:
#total N inputs in scenario after balance in kg N per year
N_inputs_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_in_incl_N_fix'])/1000
N_inputs_after_balance

In [ ]:
#should be the same as before balance
N2O_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_N2O'])/1000
N2O_after_balance

In [ ]:
NH3_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_NH3'])/1000
NH3_after_balance

In [ ]:
NOx_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_NOx'])/1000
NOx_after_balance

In [ ]:
NO3_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_NO3'])/1000
NO3_after_balance

In [ ]:
N_Harvest_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_Harvest'])/1000
N_Harvest_after_balance

In [ ]:
N_outputs_after_balance = np.sum(N_bln['Area']*(1/10000) * N_bln['N_Out'])/1000
N_outputs_after_balance

## P Balance

In [ ]:
URBAG_map.keys().values

In [ ]:
P_balance = pd.DataFrame(URBAG_map)

In [ ]:
P_balance = pd.DataFrame(URBAG_map.drop(['TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR',
                                          'TF', 'REG_CREAF', 'REG_CAT', 'TF_REG',
                                           'CAT_NIV_5', 'ORD_NIV_5', 'CAT_NIV_4', 'ORD_NIV_4', 'CAT_NIV_3',
                                           'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2', 'CAT_NIV_1F', 'ORD_NIV_1F',
                                           'CAT_NIV_1', 'ORD_NIV_1', '01_ETO', '02_ETO', '03_ETO', '04_ETO',
                                           '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO', '11_ETO',
                                           '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT', '05_PPT', '06_PPT',
                                           '07_PPT', '08_PPT', '09_PPT', '10_PPT', '11_PPT', '12_PPT',
                                           'kgP2O5/ha','N_T', 'P_T', 'K_T','01_CWR', '02_CWR',
                                           '03_CWR', '04_CWR', '05_CWR', '06_CWR', '07_CWR', '08_CWR',
                                           '09_CWR', '10_CWR', '11_CWR', '12_CWR', '01_PPTe', '02_PPTe',
                                           '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe', '07_PPTe', '08_PPTe',
                                           '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe', '01_ID', '02_ID',
                                           '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID', '09_ID',
                                           '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
                                           '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE',
                                           '11_WE', '12_WE', 'WE_m3_ha_yr',
                                            'kgK2O/ha',  'kgN/ha', 
                                           'EF_Tier2', 'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
                                           'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9', 'Kc_b_Kc10',
                                           'Kc_b_Kc11', 'Kc_b_Kc12', 'Manure', 'Manure_N', 'Manure_P2O5',
                                           'Agriwaste_toField', 'Agriwaste_burned', 'N_agriwaste_to_field'],axis=1))
P_balance.head()

### P balance before adjustments

In [ ]:
P_balance['P_Manure'] = Manure_P2O5 #Kg P2O5/ha
P_balance['P_Fertiliser'] = Mineral_fert_P2O5 #Kg P2O5/ha
P_balance['P_Struvite'] = Struvite_P2O5 #Kg P2O5/ha
P_balance['P_Compost'] = Compost_P2O5 #Kg P2O5/ha
P_balance['P_In'] = Manure_P2O5 + Mineral_fert_P2O5 + Struvite_P2O5 + Compost_P2O5
P_balance['P_PO43-'] = d_conv_emiss['PO43_runoff_water_total'] * conv9_1 #kg P2O5/ha
P_balance['P_Harvest'] = URBAG_map['kgP2O5/ha'] #P removed with harvest
P_balance['P_Out'] = P_balance['P_PO43-'] + P_balance['P_Harvest']
P_balance['P_Balance'] = P_balance['P_In'] - P_balance['P_Out'] #Kg P2O5/ha
P_balance['P_bln_check'] = np.where(P_balance['P_Balance'] >= 0, 'Enough', 'Deficient')

In [ ]:
ascending_P = P_balance.sort_values('P_Balance')
pd.unique(ascending_P['P_Balance'])

In [ ]:
P_balance.columns

In [ ]:
print("The data before balance is:")

In [ ]:
print(P_balance[['P_Manure', 'P_Fertiliser',
       'P_Struvite', 'P_Compost', 'P_In', 'P_PO43-', 'P_Harvest', 'P_Out',
       'P_Balance', 'P_bln_check']].describe())

### Adjustments via harvested P and phosphate residual emissions

In [ ]:
#Condition 1: Plots where the balance is larger than 0 i.e. inputs are larger than outputs
#Condition 2: Plots where the balance is smaller than 0 i.e. outputs are larger than inputs
cond1 = P_balance['P_Balance'] > 0
cond2 = P_balance['P_Balance'] <= 0

#For plots in condition 1:
#P fertilizer is reduced with the excess
if scen == 'S0_MinFert':
    P_balance.loc[P_balance.index[cond1],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond1],'P_Fertiliser'] - P_balance.loc[P_balance.index[cond1],'P_Balance']
elif scen == 'S0_struvite_P':
    P_balance.loc[N_balance.index[cond1],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond1],'P_Fertiliser'] - P_balance.loc[P_balance.index[cond1],'P_Balance']
elif scen == 'S0_compost':
    P_balance.loc[N_balance.index[cond1],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond1],'P_Fertiliser'] - P_balance.loc[P_balance.index[cond1],'P_Balance']
elif scen == 'S0_Ammon_salts':
    P_balance.loc[N_balance.index[cond1],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond1],'P_Fertiliser'] - P_balance.loc[P_balance.index[cond1],'P_Balance']
else:
    pass

#For plots in condition 2:
#P fertilizer is increased with the difference
if scen == 'S0_MinFert':
    P_balance.loc[P_balance.index[cond2],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond2],'P_Fertiliser'] + np.abs(P_balance.loc[P_balance.index[cond2],'P_Balance'])
elif scen == 'S0_struvite_P':
    P_balance.loc[P_balance.index[cond2],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond2],'P_Fertiliser'] + np.abs(P_balance.loc[P_balance.index[cond2],'P_Balance'])
elif scen == 'S0_compost':
    P_balance.loc[P_balance.index[cond2],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond2],'P_Fertiliser'] + np.abs(P_balance.loc[P_balance.index[cond2],'P_Balance'])
elif scen == 'S0_Ammon_salts':
    P_balance.loc[P_balance.index[cond2],'P_Fertiliser_1'] = P_balance.loc[P_balance.index[cond2],'P_Fertiliser'] + np.abs(P_balance.loc[P_balance.index[cond2],'P_Balance'])
else:
    pass

### P balance after adjustment

In [ ]:
URBAG_map.keys().values

In [ ]:
P_bln = pd.DataFrame(URBAG_map)

In [ ]:
P_bln = pd.DataFrame(URBAG_map.drop(['TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR',
                                          'TF', 'REG_CREAF', 'REG_CAT', 'TF_REG',
                                           'CAT_NIV_5', 'ORD_NIV_5', 'CAT_NIV_4', 'ORD_NIV_4', 'CAT_NIV_3',
                                           'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2', 'CAT_NIV_1F', 'ORD_NIV_1F',
                                           'CAT_NIV_1', 'ORD_NIV_1', '01_ETO', '02_ETO', '03_ETO', '04_ETO',
                                           '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO', '11_ETO',
                                           '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT', '05_PPT', '06_PPT',
                                           '07_PPT', '08_PPT', '09_PPT', '10_PPT', '11_PPT', '12_PPT',
                                           'kgP2O5/ha','N_T', 'P_T', 'K_T', '01_CWR', '02_CWR',
                                           '03_CWR', '04_CWR', '05_CWR', '06_CWR', '07_CWR', '08_CWR',
                                           '09_CWR', '10_CWR', '11_CWR', '12_CWR', '01_PPTe', '02_PPTe',
                                           '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe', '07_PPTe', '08_PPTe',
                                           '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe', '01_ID', '02_ID',
                                           '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID', '09_ID',
                                           '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
                                           '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE',
                                           '11_WE', '12_WE', 'WE_m3_ha_yr',
                                            'kgK2O/ha', 'kgN/ha', 
                                           'EF_Tier2', 'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
                                           'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9', 'Kc_b_Kc10',
                                           'Kc_b_Kc11', 'Kc_b_Kc12', 'Manure', 'Manure_N', 'Manure_P2O5',
                                           'Agriwaste_toField', 'Agriwaste_burned', 'N_agriwaste_to_field',
                                           ], axis=1))
P_bln.head()

In [ ]:
P_bln['P_Manure'] = Manure_P2O5 #Kg P2O5/ha
P_bln['P_Fertiliser'] = P_balance['P_Fertiliser_1'] #Kg P2O5/ha
P_bln['P_Struvite'] = P_balance['P_Struvite'] #Kg P2O5/ha
P_bln['P_Compost'] = Compost_P2O5 #Kg P2O5/ha
P_bln['P_In'] = P_bln['P_Manure'] + P_bln['P_Fertiliser'] + P_bln['P_Struvite'] + P_bln['P_Compost']
P_bln['P_PO43-'] = d_conv_emiss['PO43_runoff_water_total'] * conv9_1 
P_bln['P_Harvest'] = P_balance['P_Harvest'] #P removed with harvest
P_bln['P_Out'] = P_bln['P_PO43-'] + P_bln['P_Harvest'] #Total outputs
P_bln['P_Balance'] = P_bln['P_In'] - P_bln['P_Out'] #Kg P2O5/ha
P_bln['P_bln_check'] = np.where(P_bln['P_Balance'] >= 0, 'Enough', 'Deficient')

In [ ]:
print("The data after balance is:")

In [ ]:
P_bln.columns

In [ ]:
print(P_bln[['P_Manure', 'P_Fertiliser',
       'P_Struvite', 'P_Compost', 'P_In', 'P_PO43-', 'P_Harvest', 'P_Out',
       'P_Balance', 'P_bln_check']].describe())

In [ ]:
pd.unique(P_bln['P_Balance'])

In [ ]:
test = P_bln.loc[P_bln['Fid']==1882]
test['P_Fertiliser']

In [ ]:
(P_bln['P_Balance'] < 0).values.any()

In [ ]:
P_bln['P_Balance'].max()

In [ ]:
P_bln['P_Balance'].min()

In [ ]:
P_compost_after_balance = np.sum(P_bln['Area']*(1/10000) * P_bln['P_Compost'])/1000
P_compost_after_balance

In [ ]:
P_struvite_after_balance = np.sum(P_bln['Area']*(1/10000) * P_bln['P_Struvite'])/1000
P_struvite_after_balance

In [ ]:
P_MinFert_after_balance = np.sum(P_bln['Area']*(1/10000) * P_bln['P_Fertiliser'])/1000
P_MinFert_after_balance

In [ ]:
P_inputs_after_balance = np.sum(P_bln['Area']*(1/10000) * P_bln['P_In'])/1000
P_inputs_after_balance

In [ ]:
PO43_after_balance = np.sum(P_bln['Area']*(1/10000) * P_bln['P_PO43-'])/1000
PO43_after_balance

In [ ]:
P_harvest_after_balance = np.sum(P_bln['Area']*(1/10000) * P_bln['P_Harvest'])/1000
P_harvest_after_balance

In [ ]:
P_outputs_after_balance = np.sum(P_bln['Area']*(1/10000) * P_bln['P_Out'])/1000
P_outputs_after_balance

## K Balance

We assume that all K applied is uptaken by the plant and removed with the harvest.
Thus:

- K mineral fert inputs + K in compost = K removed with harvest

In [ ]:
URBAG_map.keys().values

In [ ]:
K_bln = pd.DataFrame(URBAG_map)

In [ ]:
K_bln = pd.DataFrame(URBAG_map.drop([ 'TIPO_RIEGO', 'EFIC_RIEGO', 'TIPO_DIST', 'EFIC_DISTR',
                                          'TF', 'REG_CREAF', 'REG_CAT', 'TF_REG',
                                           'CAT_NIV_5', 'ORD_NIV_5', 'CAT_NIV_4', 'ORD_NIV_4', 'CAT_NIV_3',
                                           'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2', 'CAT_NIV_1F', 'ORD_NIV_1F',
                                           'CAT_NIV_1', 'ORD_NIV_1', '01_ETO', '02_ETO', '03_ETO', '04_ETO',
                                           '05_ETO', '06_ETO', '07_ETO', '08_ETO', '09_ETO', '10_ETO', '11_ETO',
                                           '12_ETO', '01_PPT', '02_PPT', '03_PPT', '04_PPT', '05_PPT', '06_PPT',
                                           '07_PPT', '08_PPT', '09_PPT', '10_PPT', '11_PPT', '12_PPT',
                                           'kgP2O5/ha','N_T', 'P_T', 'K_T','01_CWR', '02_CWR',
                                           '03_CWR', '04_CWR', '05_CWR', '06_CWR', '07_CWR', '08_CWR',
                                           '09_CWR', '10_CWR', '11_CWR', '12_CWR', '01_PPTe', '02_PPTe',
                                           '03_PPTe', '04_PPTe', '05_PPTe', '06_PPTe', '07_PPTe', '08_PPTe',
                                           '09_PPTe', '10_PPTe', '11_PPTe', '12_PPTe', '01_ID', '02_ID',
                                           '03_ID', '04_ID', '05_ID', '06_ID', '07_ID', '08_ID', '09_ID',
                                           '10_ID', '11_ID', '12_ID', 'LOSSES', '01_WE', '02_WE', '03_WE',
                                           '04_WE', '05_WE', '06_WE', '07_WE', '08_WE', '09_WE', '10_WE',
                                           '11_WE', '12_WE', 'WE_m3_ha_yr',
                                            'kgK2O/ha',  'kgN/ha', 
                                           'EF_Tier2', 'id_final', 'Kc_b_Kc1', 'Kc_b_Kc2', 'Kc_b_Kc3', 'Kc_b_Kc4',
                                           'Kc_b_Kc5', 'Kc_b_Kc6', 'Kc_b_Kc7', 'Kc_b_Kc8', 'Kc_b_Kc9', 'Kc_b_Kc10',
                                           'Kc_b_Kc11', 'Kc_b_Kc12', 'Manure', 'Manure_N', 'Manure_P2O5',
                                           'Agriwaste_toField', 'Agriwaste_burned', 'N_agriwaste_to_field',
                                           ], axis=1))
K_bln.head()

In [ ]:
#Total K input 710.56676 tons K2O/yr
#Total K input 586.500652283024 tons K/yr, 709.665789262459 tons K2O/yr (NPK_27_06_22An - 24_02_23.xlsx)
#Total K input 586.221985694181 tons K/yr, 709.328602689959 tons K2O/yr (NPK_27_06_22An_05_03_23.xlsx)
K_bln['K_Fertiliser'] = Mineral_fert_K2O #Kg K2O/ha
K_bln['K_Compost'] = Compost_K2O #Kg K2O/ha
K_bln['K_In'] = K_bln['K_Fertiliser'] + K_bln['K_Compost']  #Kg K2O/ha
K_bln['K_Harv'] = URBAG_map['kgK2O/ha'] #Kg K2O/ha
K_bln['K_Balance'] = K_bln['K_In'] - K_bln['K_Harv'] #Kg K2O/ha

In [ ]:
pd.unique(K_bln['K_Balance'])

In [ ]:
(K_bln['K_Balance'] < 0).values.any()

In [ ]:
K_bln['K_Balance'].max()

In [ ]:
K_bln['K_Balance'].min()

In [ ]:
K_compost_after_balance = np.sum(K_bln['Area']*(1/10000) * K_bln['K_Compost'])/1000
K_compost_after_balance

In [ ]:
K_MinFert_after_balance = np.sum(K_bln['Area']*(1/10000) * K_bln['K_Fertiliser'])/1000
K_MinFert_after_balance

In [ ]:
K_in_after_balance = np.sum(K_bln['Area']*(1/10000) * K_bln['K_In'])/1000
K_in_after_balance

In [ ]:
K_harvest_after_balance = np.sum(K_bln['Area']*(1/10000) * K_bln['K_Harv'])/1000
K_harvest_after_balance

### Totals after balance for N, P and K

In [ ]:
my_N_array = np.array([[N_MinFert_after_balance,
                        N_AmmonSalts_after_balance,
                     N_struvite_after_balance,
                     N_compost_after_balance,
                     N_Fixation_after_balance,
                     N_inputs_after_balance,
                     N2O_after_balance,
                     NH3_after_balance,
                     NOx_after_balance,
                     NO3_after_balance,
                     N_Harvest_after_balance,
                     N_outputs_after_balance,
                     P_MinFert_after_balance,
                     P_struvite_after_balance,
                     P_compost_after_balance,
                     P_inputs_after_balance,
                     PO43_after_balance,
                     P_harvest_after_balance,
                     P_outputs_after_balance,
                     K_MinFert_after_balance,
                     K_compost_after_balance,   
                     K_in_after_balance,
                     K_harvest_after_balance]])

In [ ]:
Totals_after_balance = pd.DataFrame(my_N_array, columns = ['N_MinFert_NtonYR',
                                                           'N_AmmonSalts_NtonYR',
                                               'N_struvite_NtonYR',
                                               'N_compost_NtonYR',
                                               'N_fixation_NtonYR',
                                               'N_inputs_NtonYR',            
                                               'N2O_NtonYR',
                                               'NH3_NtonYR',
                                               'NOx_NtonYR',
                                               'NO3_NtonYR',
                                               'N_Harvest_NtonYR',
                                               'N_outputs_NtonYR',
                                               'P_MinFert_P205tonYR',
                                               'P_struvite_P205tonYR',
                                               'P_compost_P205tonYR',
                                               'P_inputs_P205tonYR',
                                               'PO43_P205tonYR',
                                               'P_Harvest_P2O5tonYR',
                                               'P_outputs_P205tonYR',
                                               'K_MinFert_K2OtonYR',
                                               'K_Compost_K2OtonYR',
                                               'K_inputs_K2OtonYR',
                                               "K_Harvest_K2OtonYR"])

In [ ]:
#Totals after balance, emissions and total N requirement shouold be the same as before balance. N for mineral fertilizer and struvite should change
Totals_after_balance_AMB = Totals_after_balance.T
Totals_after_balance_AMB

## Saving outputs

In [ ]:
#Nitrogen balance in kg N/ha yr
Nitrogen_Balance = pd.DataFrame(N_bln)

#P balance in kg P2O5/ha yr
Phosphorus_Balance = pd.DataFrame(P_bln)

#K balance in kg K2O/ha yr
Potassium_Balance = pd.DataFrame(K_bln)

In [ ]:
Nitrogen_Balance.keys()

In [ ]:
Phosphorus_Balance.keys()

In [ ]:
Potassium_Balance.keys()

In [ ]:
URBAG_map.keys()

In [ ]:
#kg emission /ha year
%store d_conv_emiss 
%store Nitrogen_Balance
%store Phosphorus_Balance
%store Potassium_Balance
%store URBAG_map

In [ ]:
#kg emission /ha year
d_conv_emiss.to_csv("output/" + scen + "/emissions/LET_emissions_after_bln.csv")

In [ ]:
#Nitrogen balance in kg N/ha yr
Nitrogen_Balance.to_csv("output/" + scen + "/balances/Nitrogen_Balance_new.csv")

In [ ]:
#P balance in kg P2O5/ha yr
Phosphorus_Balance.to_csv("output/" + scen + "/balances/Phosphorus_Balance_New.csv")

In [ ]:
#K balance in kg K2O/ha yr
Potassium_Balance.to_csv("output/" + scen + "/balances/Potassium_Balance_New.csv")

In [ ]:
Totals_after_balance_AMB.to_csv("output/" + scen + "/balances/Totals_after_balance.csv")

## VISUALIZATION

# Nitrogen balance

In [ ]:
Nitrogen_Balance.keys()

In [ ]:
N_balance_boxplot = Nitrogen_Balance.drop(['DUN_CULTIU', 'Voronoi_1', 'CAT_NIV_5', 'ORD_NIV_5', 'CAT_NIV_4',
       'ORD_NIV_4', 'CAT_NIV_3', 'ORD_NIV_3', 'CAT_NIV_2', 'ORD_NIV_2',
       'CAT_NIV_1F', 'ORD_NIV_1F', 'CAT_NIV_1', 'ORD_NIV_1', 'ID_Voronoi',
       'Municipi', 'Codi_munic', 'Area', 'Fid', 'Comarca', 'Codi_comar',
       'ID_CUENCA', 'CUENCA', 'yield_kgHA', 'kgP2O5/ha', 'kgK2O/ha', 'kgN/ha','N_In','N_Balance',
       'N_bln_check',
       ], axis=1)

In [ ]:
N_balance_boxplot1 = []
for x in N_balance_boxplot.keys():
    print(x)
    N_balance_boxplot1.append(N_balance_boxplot[x])

In [ ]:
fig, axs = plt.subplots(5, 3, figsize=(10,8))
axs[0, 0].boxplot(N_balance_boxplot1[0], 0, '', labels = [''], showmeans=True)
axs[0, 0].set_title("Manure")
axs[0, 1].boxplot(N_balance_boxplot1[1], 0, '', labels = [''], showmeans=True)
axs[0, 1].set_title("Mineral Fertiliser")
axs[0, 2].boxplot(N_balance_boxplot1[2], 0, '', labels = [''], showmeans=True)
axs[0, 2].set_title("Agricultural Residues")
axs[1, 0].boxplot(N_balance_boxplot1[3], 0, '', labels = [''], showmeans=True)
axs[1, 0].set_title("Struvite")
axs[1, 1].boxplot(N_balance_boxplot1[4], 0, '', labels = [''], showmeans=True)
axs[1, 1].set_title("Compost")
axs[1, 2].boxplot(N_balance_boxplot1[5], 0, labels = [''], showmeans=True)
axs[1, 2].set_title("Ammonium Salts")
axs[2, 0].boxplot(N_balance_boxplot1[6], 0, '', labels = [''], showmeans=True)
axs[2, 0].set_title("N Fixation") #
axs[2, 1].boxplot(N_balance_boxplot1[7], 0, '', labels = [''], showmeans=True)
axs[2, 1].set_title("Total N inputs incl. fixation")
axs[2, 2].boxplot(N_balance_boxplot1[8], 0, '', labels = [''], showmeans=True)
axs[2, 2].set_title("NH$_{3}$ emissions")
axs[3, 0].boxplot(N_balance_boxplot1[9], 0, '', labels = [''], showmeans=True)
axs[3, 0].set_title("NO$_{x}$ emissions")
axs[3, 1].boxplot(N_balance_boxplot1[10], 0, '', labels = [''], showmeans=True)
axs[3, 1].set_title("N$_{2}$O emissions")
axs[3, 2].boxplot(N_balance_boxplot1[11], 0, labels = [''], showmeans=True)
axs[3, 2].set_title("NO$_{3}^-$ emissions")
axs[4, 0].boxplot(N_balance_boxplot1[12], 0, '', labels = [''], showmeans=True)
axs[4, 0].set_title("N removed with Harvest")
axs[4, 1].boxplot(N_balance_boxplot1[13], 0, '', labels = [''], showmeans=True)
axs[4, 1].set_title("Total N output")
fig.delaxes(axs[4,2])
fig.suptitle("N inputs and outputs - kg N/ha yr")
fig.tight_layout()
#fig.savefig( 'output/' + scen + '/emissions/N_balance' + '.png')

# Phosphorus balance

In [ ]:
Phosphorus_Balance.keys()

In [ ]:
P_balance_boxplot = Phosphorus_Balance.drop(['DUN_GRUP', 'DUN_CULTIU', 'Voronoi_1', 'ID_Voronoi', 'Municipi',
       'Codi_munic', 'Area', 'Fid', 'Comarca', 'Codi_comar', 'ID_CUENCA',
       'CUENCA', 'ID_SUBCUEN', 'N_SUBCUENC', 'SUBCUENCAS',
       'yield_kgHA', 'Voronoi_ag', 'Vor_ag_Num','FoodTons_N','P_Balance', 'P_bln_check'], axis=1)

In [ ]:
P_balance_boxplot1 = []
for x in P_balance_boxplot.keys():
    print(x)
    P_balance_boxplot1.append(P_balance_boxplot[x])

In [ ]:
fig, axs = plt.subplots(2, 4, figsize=(10,6))
axs[0, 0].boxplot(P_balance_boxplot1[0], 0, '', labels = [''], showmeans=True)
axs[0, 0].set_title("Manure")
axs[0, 1].boxplot(P_balance_boxplot1[1], 0, '', labels = [''], showmeans=True)
axs[0, 1].set_title("Mineral Fertiliser")
axs[0, 2].boxplot(P_balance_boxplot1[2], 0, '', labels = [''], showmeans=True)
axs[0, 2].set_title("Struvite")
axs[0, 3].boxplot(P_balance_boxplot1[3], 0, '', labels = [''], showmeans=True)
axs[0, 3].set_title("Compost")
axs[1, 0].boxplot(P_balance_boxplot1[4], 0, '', labels = [''], showmeans=True)
axs[1, 0].set_title("Total P inputs")
axs[1, 1].boxplot(P_balance_boxplot1[5], 0, '', labels = [''], showmeans=True)
axs[1, 1].set_title("PO$_{4}^3-$ runoff")
axs[1, 2].boxplot(P_balance_boxplot1[6], 0, labels = [''], showmeans=True)
axs[1, 2].set_title("P removed \n with harvest")
axs[1, 3].boxplot(P_balance_boxplot1[7], 0, '', labels = [''], showmeans=True)
axs[1, 3].set_title("Total P output")
fig.suptitle("P inputs and outputs - kg P2O5/ha yr")
fig.tight_layout()
#fig.savefig( 'output/' + scen + '/emissions/P_balance' + '.png')

# Potassium balance

In [ ]:
Potassium_Balance.keys()

In [ ]:
K_balance_boxplot = Potassium_Balance.drop(['DUN_GRUP', 'DUN_CULTIU', 'Voronoi_1', 'ID_Voronoi', 'Municipi',
       'Codi_munic', 'Area', 'Fid', 'Comarca', 'Codi_comar', 'ID_CUENCA',
       'CUENCA', 'ID_SUBCUEN', 'N_SUBCUENC', 'SUBCUENCAS',
       'yield_kgHA', 'Voronoi_ag', 'Vor_ag_Num', 'FoodTons_N','K_Balance'], axis=1)

In [ ]:
K_balance_boxplot1 = []
for x in K_balance_boxplot.keys():
    print(x)
    K_balance_boxplot1.append(K_balance_boxplot[x])

In [ ]:
fig, axs = plt.subplots(1, 4, figsize=(10,4))
axs[0].boxplot(K_balance_boxplot1[0], 0, '', labels = [''], showmeans=True)
axs[0].set_title("Mineral Fertilizer")
axs[1].boxplot(K_balance_boxplot1[1], 0, '', labels = [''], showmeans=True)
axs[1].set_title("Compost")
axs[2].boxplot(K_balance_boxplot1[2], 0, '', labels = [''], showmeans=True)
axs[2].set_title("Total inputs")
axs[3].boxplot(K_balance_boxplot1[3], 0, '', labels = [''], showmeans=True)
axs[3].set_title("K removed with harvest")
fig.suptitle("K inputs and outputs - kg K2O/ha yr")
fig.tight_layout()
#fig.savefig( 'output/' + scen + '/emissions/K_balance' + '.png')